# Model Training Notebook

This notebook trains the three CNN Architectures (ResNet, DenseNet, ConvNeXt), each under 4 different augmentation strategies.


In [109]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pathlib import Path
import time
import copy
import os
import random

In [110]:
torch.__version__

'2.9.0+cu128'

In [111]:
#data_paths
 
input_path = Path("data/datasets/trashnet")
input_path2 = Path("data/datasets/trashvariety")

In [112]:
#function to save models later on

model_dir = "trained_models"
os.makedirs(model_dir, exist_ok=True)

def save_model(model, model_name, history=None):
    save_path = os.path.join(model_dir, f"{model_name}.pth")

    checkpoint = {
        "model_state_dict": model.state_dict(),
    }

    if history is not None:
        checkpoint["history"] = history

    torch.save(checkpoint, save_path)
    print(f"Model saved")    

In [113]:
seed = 64

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [114]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [115]:
#calculate class weighted cross entropy loss

from collections import Counter

def compute_class_weights_from_imagefolder(train_root, num_classes):
    ds = datasets.ImageFolder(train_root, transform=transforms.ToTensor())
    counts = Counter(ds.targets)
    total = sum(counts.values())

    weights = [total / counts[i] for i in range(num_classes)]
    weights = torch.tensor(weights, dtype=torch.float)
    weights = weights / weights.sum() * num_classes 
    return weights, ds.classes, counts

tmp_ds = datasets.ImageFolder(input_path / "train")
num_classes = len(tmp_ds.classes)

class_weights, class_names, train_counts = compute_class_weights_from_imagefolder(
    input_path / "train",
    num_classes
)

class_weights = class_weights.to(device)

criterion_weighted = torch.nn.CrossEntropyLoss(weight=class_weights)

print("Classes:", class_names)
print("Train counts:", train_counts)
print("Class weights:", class_weights)

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Train counts: Counter({3: 475, 1: 400, 4: 385, 2: 328, 0: 322, 5: 109})
Class weights: tensor([0.8270, 0.6657, 0.8119, 0.5606, 0.6917, 2.4431], device='cuda:0')


## Data Augmentation data transformations

In [116]:
# baseline data transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {

    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),  
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),    
}

image_datasets = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms["val"]),
}


dataloaders = {
    "train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets["test"], batch_size=32, shuffle=False, num_workers=4),
}   


dataset_size = {
    "train": len(image_datasets["train"]),
    "val": len(image_datasets["val"]),
}

num_classes = len((image_datasets["train"]).classes)

In [117]:
#geometric augmentation

data_transforms_geo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_geo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_geo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_geo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_geo["val"]),
}

dataloaders_geo = {
    "train": torch.utils.data.DataLoader(image_datasets_geo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_geo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_geo["test"], batch_size=32, shuffle=False, num_workers=4),
}

dataset_size_geo = {
    "train": len(image_datasets_geo["train"]),
    "val": len(image_datasets_geo["val"]),
    "test": len(image_datasets_geo["test"]),
}

num_classes_geo = len(image_datasets_geo["train"].classes)


In [118]:
#photometric
 
data_transforms_photo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_photo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_photo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_photo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_photo["val"]),
}

dataloaders_photo = {
    "train": torch.utils.data.DataLoader(image_datasets_photo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_photo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_photo["test"], batch_size=32, shuffle=False, num_workers=4),
}

dataset_size_photo = {
    "train": len(image_datasets_photo["train"]),
    "val": len(image_datasets_photo["val"]),
    "test": len(image_datasets_photo["test"]),
}

num_classes_photo = len(image_datasets_photo["train"].classes)


In [119]:
#aug mix
 
data_transforms_mix = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        normalize,
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value='random')
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_mix = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_mix["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_mix["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_mix["val"]),
}

dataloaders_mix = {
    "train": torch.utils.data.DataLoader(image_datasets_mix["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_mix["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_mix["test"], batch_size=32, shuffle=False, num_workers=4),
}

dataset_size_mix = {
    "train": len(image_datasets_mix["train"]),
    "val": len(image_datasets_mix["val"]),
    "test": len(image_datasets_mix["test"]),
}

num_classes_mix = len(image_datasets_mix["train"].classes)


## Model Training function

In [120]:
def train_model(model, dataloaders, dataset_size, device,
                criterion, optimizer, num_epochs=25):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels).item()

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects / dataset_size[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "train":
                history["train_loss"].append(epoch_loss)
                history["train_acc"].append(epoch_acc)
            else:
                history["val_loss"].append(epoch_loss)
                history["val_acc"].append(epoch_acc)

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model, history


## ResNet50 training 

In [121]:
#RESNET-50

resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes) #Replace last classifier layer
resnet = resnet.to(device)

optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet, resnet_history = train_model(
    model=resnet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6375 Acc: 0.4611
val Loss: 1.3684 Acc: 0.6813

Epoch 2/25
----------
train Loss: 1.2056 Acc: 0.6667
val Loss: 0.9542 Acc: 0.7331

Epoch 3/25
----------
train Loss: 0.8874 Acc: 0.7276
val Loss: 0.6924 Acc: 0.8048

Epoch 4/25
----------
train Loss: 0.7339 Acc: 0.7543
val Loss: 0.5887 Acc: 0.8048

Epoch 5/25
----------
train Loss: 0.5982 Acc: 0.7999
val Loss: 0.4649 Acc: 0.8685

Epoch 6/25
----------
train Loss: 0.5391 Acc: 0.8222
val Loss: 0.4181 Acc: 0.8765

Epoch 7/25
----------
train Loss: 0.4796 Acc: 0.8484
val Loss: 0.3751 Acc: 0.8845

Epoch 8/25
----------
train Loss: 0.4100 Acc: 0.8628
val Loss: 0.3304 Acc: 0.9084

Epoch 9/25
----------
train Loss: 0.3720 Acc: 0.8732
val Loss: 0.3182 Acc: 0.8964

Epoch 10/25
----------
train Loss: 0.3624 Acc: 0.8891
val Loss: 0.2875 Acc: 0.9124

Epoch 11/25
----------
train Loss: 0.3082 Acc: 0.8945
val Loss: 0.2730 Acc: 0.9044

Epoch 12/25
----------
train Loss: 0.2818 Acc: 0.9158
val Loss: 0.2734 Acc: 0.9124

E

In [122]:
save_model(resnet, "resnet50_baseline", resnet_history)

del resnet
del optimizer
del resnet_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [123]:
#RESNET-50 photo augmentations

resnet_aug_1 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_1.fc = nn.Linear(resnet_aug_1.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_1 = resnet_aug_1.to(device)

optimizer = optim.SGD(resnet_aug_1.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_aug_1, resnet_history_aug_1 = train_model(
    model=resnet_aug_1,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6501 Acc: 0.4388
val Loss: 1.3872 Acc: 0.6813

Epoch 2/25
----------
train Loss: 1.2802 Acc: 0.6360
val Loss: 0.9720 Acc: 0.7092

Epoch 3/25
----------
train Loss: 0.9809 Acc: 0.6999
val Loss: 0.7083 Acc: 0.7849

Epoch 4/25
----------
train Loss: 0.7928 Acc: 0.7434
val Loss: 0.6124 Acc: 0.8048

Epoch 5/25
----------
train Loss: 0.6941 Acc: 0.7702
val Loss: 0.4821 Acc: 0.8486

Epoch 6/25
----------
train Loss: 0.5982 Acc: 0.8029
val Loss: 0.4811 Acc: 0.8486

Epoch 7/25
----------
train Loss: 0.5671 Acc: 0.7979
val Loss: 0.4064 Acc: 0.8685

Epoch 8/25
----------
train Loss: 0.5154 Acc: 0.8222
val Loss: 0.3598 Acc: 0.8924

Epoch 9/25
----------
train Loss: 0.4638 Acc: 0.8455
val Loss: 0.3537 Acc: 0.8884

Epoch 10/25
----------
train Loss: 0.4428 Acc: 0.8564
val Loss: 0.3120 Acc: 0.9203

Epoch 11/25
----------
train Loss: 0.4045 Acc: 0.8668
val Loss: 0.3341 Acc: 0.9084

Epoch 12/25
----------
train Loss: 0.4086 Acc: 0.8633
val Loss: 0.2821 Acc: 0.9163

E

In [124]:
save_model(resnet_aug_1, "resnet50_photo", resnet_history_aug_1)

del resnet_aug_1
del optimizer
del resnet_history_aug_1
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [125]:
#RESNET-50 geo augmentations

resnet_aug_2 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_2.fc = nn.Linear(resnet_aug_2.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_2 = resnet_aug_2.to(device)

optimizer = optim.SGD(resnet_aug_2.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_aug_2, resnet_history_aug_2 = train_model(
    model=resnet_aug_2,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6786 Acc: 0.4255
val Loss: 1.4738 Acc: 0.6494

Epoch 2/25
----------
train Loss: 1.2989 Acc: 0.6261
val Loss: 1.1055 Acc: 0.7131

Epoch 3/25
----------
train Loss: 0.9832 Acc: 0.6934
val Loss: 0.8135 Acc: 0.7729

Epoch 4/25
----------
train Loss: 0.8209 Acc: 0.7281
val Loss: 0.6468 Acc: 0.8207

Epoch 5/25
----------
train Loss: 0.6740 Acc: 0.7756
val Loss: 0.5701 Acc: 0.8247

Epoch 6/25
----------
train Loss: 0.5846 Acc: 0.8063
val Loss: 0.4976 Acc: 0.8088

Epoch 7/25
----------
train Loss: 0.5354 Acc: 0.8088
val Loss: 0.4325 Acc: 0.8446

Epoch 8/25
----------
train Loss: 0.4913 Acc: 0.8321
val Loss: 0.3938 Acc: 0.8566

Epoch 9/25
----------
train Loss: 0.4378 Acc: 0.8415
val Loss: 0.3832 Acc: 0.8725

Epoch 10/25
----------
train Loss: 0.3976 Acc: 0.8702
val Loss: 0.3321 Acc: 0.9004

Epoch 11/25
----------
train Loss: 0.3834 Acc: 0.8658
val Loss: 0.3427 Acc: 0.8884

Epoch 12/25
----------
train Loss: 0.3525 Acc: 0.8732
val Loss: 0.3116 Acc: 0.9124

E

In [126]:
save_model(resnet_aug_2, "resnet50_geo", resnet_history_aug_2)

del resnet_aug_2
del optimizer
del resnet_history_aug_2
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [127]:
#RESNET-50 mixed augmentations

resnet_mixed = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_mixed.fc = nn.Linear(resnet_mixed.fc.in_features, num_classes) #Replace last classifier layer
resnet_mixed = resnet_mixed.to(device)

optimizer = optim.SGD(resnet_mixed.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_mixed, resnet_mixed_history = train_model(
    model=resnet_mixed,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.7194 Acc: 0.3531
val Loss: 1.5797 Acc: 0.5896

Epoch 2/25
----------
train Loss: 1.4742 Acc: 0.5924
val Loss: 1.2368 Acc: 0.6614

Epoch 3/25
----------
train Loss: 1.1943 Acc: 0.6112
val Loss: 0.9782 Acc: 0.7092

Epoch 4/25
----------
train Loss: 1.0066 Acc: 0.6741
val Loss: 0.7668 Acc: 0.7729

Epoch 5/25
----------
train Loss: 0.8706 Acc: 0.7132
val Loss: 0.6563 Acc: 0.7928

Epoch 6/25
----------
train Loss: 0.7809 Acc: 0.7464
val Loss: 0.5950 Acc: 0.8327

Epoch 7/25
----------
train Loss: 0.6902 Acc: 0.7642
val Loss: 0.5066 Acc: 0.8367

Epoch 8/25
----------
train Loss: 0.6546 Acc: 0.7895
val Loss: 0.4954 Acc: 0.8287

Epoch 9/25
----------
train Loss: 0.6152 Acc: 0.7860
val Loss: 0.4562 Acc: 0.8327

Epoch 10/25
----------
train Loss: 0.5631 Acc: 0.8049
val Loss: 0.4487 Acc: 0.8526

Epoch 11/25
----------
train Loss: 0.5018 Acc: 0.8192
val Loss: 0.3875 Acc: 0.8645

Epoch 12/25
----------
train Loss: 0.5089 Acc: 0.8197
val Loss: 0.4112 Acc: 0.8645

E

In [128]:
save_model(resnet_mixed, "resnet50_mixed", resnet_mixed_history)

del resnet_mixed
del optimizer
del resnet_mixed_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

## DenseNet-121 training

In [129]:
#densenet

densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet = densenet.to(device)

optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet, densenet_history = train_model(
    model=densenet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.2658 Acc: 0.5389
val Loss: 0.7147 Acc: 0.7809

Epoch 2/25
----------
train Loss: 0.6911 Acc: 0.7677
val Loss: 0.4956 Acc: 0.8486

Epoch 3/25
----------
train Loss: 0.5305 Acc: 0.8158
val Loss: 0.4157 Acc: 0.8725

Epoch 4/25
----------
train Loss: 0.4111 Acc: 0.8618
val Loss: 0.3513 Acc: 0.8924

Epoch 5/25
----------
train Loss: 0.3520 Acc: 0.8777
val Loss: 0.2959 Acc: 0.8805

Epoch 6/25
----------
train Loss: 0.3002 Acc: 0.8970
val Loss: 0.2830 Acc: 0.8964

Epoch 7/25
----------
train Loss: 0.2819 Acc: 0.9054
val Loss: 0.2807 Acc: 0.9044

Epoch 8/25
----------
train Loss: 0.2598 Acc: 0.9118
val Loss: 0.2834 Acc: 0.8964

Epoch 9/25
----------
train Loss: 0.2234 Acc: 0.9257
val Loss: 0.2715 Acc: 0.9004

Epoch 10/25
----------
train Loss: 0.1999 Acc: 0.9287
val Loss: 0.2415 Acc: 0.9283

Epoch 11/25
----------
train Loss: 0.1848 Acc: 0.9381
val Loss: 0.2387 Acc: 0.9163

Epoch 12/25
----------
train Loss: 0.1959 Acc: 0.9391
val Loss: 0.2309 Acc: 0.9283

E

In [130]:
save_model(densenet, "densenet121_baseline", densenet_history)

del densenet
del optimizer
del densenet_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [131]:
#densenet geometric augmentation

densenet_geo = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_geo.classifier = nn.Linear(densenet_geo.classifier.in_features, num_classes)
densenet_geo = densenet_geo.to(device)

optimizer = optim.SGD(densenet_geo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_geo, densenet_geo_history = train_model(
    model=densenet_geo,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3699 Acc: 0.5017
val Loss: 0.8682 Acc: 0.7131

Epoch 2/25
----------
train Loss: 0.8207 Acc: 0.7018
val Loss: 0.5602 Acc: 0.8247

Epoch 3/25
----------
train Loss: 0.6580 Acc: 0.7662
val Loss: 0.4500 Acc: 0.8406

Epoch 4/25
----------
train Loss: 0.5358 Acc: 0.8148
val Loss: 0.4373 Acc: 0.8486

Epoch 5/25
----------
train Loss: 0.4694 Acc: 0.8474
val Loss: 0.3638 Acc: 0.8566

Epoch 6/25
----------
train Loss: 0.4147 Acc: 0.8544
val Loss: 0.3193 Acc: 0.8845

Epoch 7/25
----------
train Loss: 0.3895 Acc: 0.8583
val Loss: 0.3824 Acc: 0.8406

Epoch 8/25
----------
train Loss: 0.3606 Acc: 0.8806
val Loss: 0.3582 Acc: 0.8845

Epoch 9/25
----------
train Loss: 0.3166 Acc: 0.8841
val Loss: 0.3345 Acc: 0.8645

Epoch 10/25
----------
train Loss: 0.3046 Acc: 0.8950
val Loss: 0.2698 Acc: 0.8805

Epoch 11/25
----------
train Loss: 0.2640 Acc: 0.9024
val Loss: 0.3231 Acc: 0.8845

Epoch 12/25
----------
train Loss: 0.2679 Acc: 0.9099
val Loss: 0.3344 Acc: 0.8685

E

In [132]:
save_model(densenet_geo, "densenet121_geo", densenet_geo_history)

del densenet_geo
del optimizer
del densenet_geo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [133]:
#densenet photometric augmentation

densenet_photo = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_photo.classifier = nn.Linear(densenet_photo.classifier.in_features, num_classes)
densenet_photo = densenet_photo.to(device)

optimizer = optim.SGD(densenet_photo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_photo, densenet_photo_history = train_model(
    model=densenet_photo,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3406 Acc: 0.5042
val Loss: 0.7244 Acc: 0.7769

Epoch 2/25
----------
train Loss: 0.7948 Acc: 0.7434
val Loss: 0.4802 Acc: 0.8566

Epoch 3/25
----------
train Loss: 0.6232 Acc: 0.7821
val Loss: 0.4057 Acc: 0.8725

Epoch 4/25
----------
train Loss: 0.5204 Acc: 0.8237
val Loss: 0.3461 Acc: 0.8884

Epoch 5/25
----------
train Loss: 0.4789 Acc: 0.8415
val Loss: 0.3411 Acc: 0.8845

Epoch 6/25
----------
train Loss: 0.3728 Acc: 0.8697
val Loss: 0.2775 Acc: 0.8884

Epoch 7/25
----------
train Loss: 0.3770 Acc: 0.8658
val Loss: 0.2855 Acc: 0.8884

Epoch 8/25
----------
train Loss: 0.3240 Acc: 0.8826
val Loss: 0.2795 Acc: 0.8805

Epoch 9/25
----------
train Loss: 0.3011 Acc: 0.8935
val Loss: 0.2897 Acc: 0.9004

Epoch 10/25
----------
train Loss: 0.3045 Acc: 0.8965
val Loss: 0.2832 Acc: 0.8845

Epoch 11/25
----------
train Loss: 0.2841 Acc: 0.8945
val Loss: 0.2168 Acc: 0.9084

Epoch 12/25
----------
train Loss: 0.2867 Acc: 0.8920
val Loss: 0.2735 Acc: 0.9044

E

In [134]:
save_model(densenet_photo, "densenet121_photo", densenet_photo_history)

del densenet_photo
del optimizer
del densenet_photo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [135]:
#densenet mixed augmentation

densenet_mix = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_mix.classifier = nn.Linear(densenet_mix.classifier.in_features, num_classes)
densenet_mix = densenet_mix.to(device)

optimizer = optim.SGD(densenet_mix.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_mix, densenet_mix_history = train_model(
    model=densenet_mix,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.4985 Acc: 0.4156
val Loss: 1.0050 Acc: 0.6375

Epoch 2/25
----------
train Loss: 0.9858 Acc: 0.6355
val Loss: 0.6552 Acc: 0.7809

Epoch 3/25
----------
train Loss: 0.7769 Acc: 0.7132
val Loss: 0.5078 Acc: 0.8367

Epoch 4/25
----------
train Loss: 0.6567 Acc: 0.7558
val Loss: 0.4517 Acc: 0.8287

Epoch 5/25
----------
train Loss: 0.5912 Acc: 0.7895
val Loss: 0.3909 Acc: 0.8606

Epoch 6/25
----------
train Loss: 0.5656 Acc: 0.7949
val Loss: 0.3669 Acc: 0.8645

Epoch 7/25
----------
train Loss: 0.5149 Acc: 0.8153
val Loss: 0.4079 Acc: 0.8486

Epoch 8/25
----------
train Loss: 0.4959 Acc: 0.8237
val Loss: 0.3543 Acc: 0.8566

Epoch 9/25
----------
train Loss: 0.4642 Acc: 0.8331
val Loss: 0.3604 Acc: 0.8765

Epoch 10/25
----------
train Loss: 0.4208 Acc: 0.8450
val Loss: 0.3141 Acc: 0.8805

Epoch 11/25
----------
train Loss: 0.3926 Acc: 0.8574
val Loss: 0.3024 Acc: 0.8805

Epoch 12/25
----------
train Loss: 0.3395 Acc: 0.8856
val Loss: 0.3154 Acc: 0.8805

E

In [136]:
save_model(densenet_mix, "densenet121_mix", densenet_mix_history)

del densenet_mix
del optimizer
del densenet_mix_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

# ConvNeXt tiny training

In [137]:
# ConvNeXt-Base
convnext_base = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_base.classifier[2].in_features
convnext_base.classifier[2] = nn.Linear(in_features, num_classes)
convnext_base = convnext_base.to(device)

optimizer = optim.SGD(convnext_base.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_base, convnext_base_history = train_model(
    model=convnext_base,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3264 Acc: 0.5106
val Loss: 0.7841 Acc: 0.7769

Epoch 2/25
----------
train Loss: 0.7836 Acc: 0.7420
val Loss: 0.7604 Acc: 0.7131

Epoch 3/25
----------
train Loss: 0.7734 Acc: 0.7385
val Loss: 0.8314 Acc: 0.7131

Epoch 4/25
----------
train Loss: 0.5952 Acc: 0.7974
val Loss: 0.5983 Acc: 0.7849

Epoch 5/25
----------
train Loss: 0.5506 Acc: 0.8128
val Loss: 0.4095 Acc: 0.8606

Epoch 6/25
----------
train Loss: 0.4706 Acc: 0.8405
val Loss: 0.4024 Acc: 0.8486

Epoch 7/25
----------
train Loss: 0.3643 Acc: 0.8891
val Loss: 0.4223 Acc: 0.8566

Epoch 8/25
----------
train Loss: 0.4044 Acc: 0.8569
val Loss: 0.3819 Acc: 0.8765

Epoch 9/25
----------
train Loss: 0.3454 Acc: 0.8876
val Loss: 0.3060 Acc: 0.8924

Epoch 10/25
----------
train Loss: 0.3091 Acc: 0.8905
val Loss: 0.2669 Acc: 0.8964

Epoch 11/25
----------
train Loss: 0.2781 Acc: 0.9009
val Loss: 0.2763 Acc: 0.9124

Epoch 12/25
----------
train Loss: 0.2944 Acc: 0.9019
val Loss: 0.2702 Acc: 0.9004

E

In [138]:
save_model(convnext_base, "convnext_tiny_base", convnext_base_history)

del convnext_base
del optimizer
del convnext_base_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [139]:
# ConvNeXt-geo
convnext_geo = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_geo.classifier[2].in_features
convnext_geo.classifier[2] = nn.Linear(in_features, num_classes)
convnext_geo = convnext_geo.to(device)

optimizer = optim.SGD(convnext_geo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_geo, convnext_geo_history = train_model(
    model=convnext_geo,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.2981 Acc: 0.5394
val Loss: 0.7580 Acc: 0.8008

Epoch 2/25
----------
train Loss: 0.8527 Acc: 0.7078
val Loss: 0.6805 Acc: 0.7928

Epoch 3/25
----------
train Loss: 0.7569 Acc: 0.7340
val Loss: 0.5827 Acc: 0.7968

Epoch 4/25
----------
train Loss: 0.6629 Acc: 0.7717
val Loss: 0.4629 Acc: 0.8406

Epoch 5/25
----------
train Loss: 0.5377 Acc: 0.8192
val Loss: 0.4990 Acc: 0.8287

Epoch 6/25
----------
train Loss: 0.5105 Acc: 0.8257
val Loss: 0.4655 Acc: 0.8606

Epoch 7/25
----------
train Loss: 0.4963 Acc: 0.8276
val Loss: 0.3952 Acc: 0.8765

Epoch 8/25
----------
train Loss: 0.4191 Acc: 0.8579
val Loss: 0.3756 Acc: 0.8765

Epoch 9/25
----------
train Loss: 0.4017 Acc: 0.8628
val Loss: 0.3764 Acc: 0.8685

Epoch 10/25
----------
train Loss: 0.3426 Acc: 0.8772
val Loss: 0.3103 Acc: 0.8765

Epoch 11/25
----------
train Loss: 0.3566 Acc: 0.8712
val Loss: 0.4350 Acc: 0.8486

Epoch 12/25
----------
train Loss: 0.3417 Acc: 0.8876
val Loss: 0.3624 Acc: 0.8725

E

In [140]:
save_model(convnext_geo, "convnext_tiny_geo", convnext_geo_history)

del convnext_geo
del optimizer
del convnext_geo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [141]:
# ConvNeXt-photo
convnext_photo = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_photo.classifier[2].in_features
convnext_photo.classifier[2] = nn.Linear(in_features, num_classes)
convnext_photo = convnext_photo.to(device)

optimizer = optim.SGD(convnext_photo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_photo, convnext_photo_history = train_model(
    model=convnext_photo,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.2895 Acc: 0.5324
val Loss: 0.7287 Acc: 0.7689

Epoch 2/25
----------
train Loss: 0.8113 Acc: 0.7221
val Loss: 0.5866 Acc: 0.8127

Epoch 3/25
----------
train Loss: 0.7447 Acc: 0.7355
val Loss: 0.5323 Acc: 0.8287

Epoch 4/25
----------
train Loss: 0.5973 Acc: 0.7900
val Loss: 0.4692 Acc: 0.8247

Epoch 5/25
----------
train Loss: 0.5667 Acc: 0.8049
val Loss: 0.4150 Acc: 0.8645

Epoch 6/25
----------
train Loss: 0.5527 Acc: 0.8058
val Loss: 0.4643 Acc: 0.8167

Epoch 7/25
----------
train Loss: 0.4596 Acc: 0.8316
val Loss: 0.3357 Acc: 0.8805

Epoch 8/25
----------
train Loss: 0.4567 Acc: 0.8296
val Loss: 0.3827 Acc: 0.8566

Epoch 9/25
----------
train Loss: 0.4399 Acc: 0.8465
val Loss: 0.3673 Acc: 0.8725

Epoch 10/25
----------
train Loss: 0.3935 Acc: 0.8658
val Loss: 0.3286 Acc: 0.8725

Epoch 11/25
----------
train Loss: 0.3485 Acc: 0.8722
val Loss: 0.3315 Acc: 0.8805

Epoch 12/25
----------
train Loss: 0.3654 Acc: 0.8747
val Loss: 0.3932 Acc: 0.8805

E

In [142]:
save_model(convnext_photo, "convnext_tiny_photo", convnext_photo_history)

del convnext_photo
del optimizer
del convnext_photo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [143]:
# ConvNeXt-mixed
convnext_mixed = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_mixed.classifier[2].in_features
convnext_mixed.classifier[2] = nn.Linear(in_features, num_classes)
convnext_mixed = convnext_mixed.to(device)

optimizer = optim.SGD(convnext_mixed.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_mixed, convnext_mixed_history = train_model(
    model=convnext_mixed,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.5199 Acc: 0.4106
val Loss: 1.0085 Acc: 0.6534

Epoch 2/25
----------
train Loss: 0.9788 Acc: 0.6543
val Loss: 0.7673 Acc: 0.7689

Epoch 3/25
----------
train Loss: 0.8338 Acc: 0.7127
val Loss: 0.5620 Acc: 0.8327

Epoch 4/25
----------
train Loss: 0.7217 Acc: 0.7568
val Loss: 0.5186 Acc: 0.8327

Epoch 5/25
----------
train Loss: 0.7161 Acc: 0.7524
val Loss: 0.5239 Acc: 0.8367

Epoch 6/25
----------
train Loss: 0.6861 Acc: 0.7558
val Loss: 0.5033 Acc: 0.8406

Epoch 7/25
----------
train Loss: 0.6059 Acc: 0.7831
val Loss: 0.4258 Acc: 0.8606

Epoch 8/25
----------
train Loss: 0.5250 Acc: 0.8207
val Loss: 0.5127 Acc: 0.8327

Epoch 9/25
----------
train Loss: 0.5447 Acc: 0.8044
val Loss: 0.4419 Acc: 0.8486

Epoch 10/25
----------
train Loss: 0.4948 Acc: 0.8311
val Loss: 0.3742 Acc: 0.8725

Epoch 11/25
----------
train Loss: 0.4795 Acc: 0.8252
val Loss: 0.3553 Acc: 0.8924

Epoch 12/25
----------
train Loss: 0.4422 Acc: 0.8346
val Loss: 0.3483 Acc: 0.8765

E

In [144]:
save_model(convnext_mixed, "convnext_tiny_mixed", convnext_mixed_history)

del convnext_mixed
del optimizer
del convnext_mixed_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>